In [1]:
import causal_nf.utils.wandb_local as wandb_local
import causal_nf.config as causal_nf_config
from causal_nf.config import cfg
import causal_nf.utils.training as causal_nf_train
import os
import glob
import argparse
from yacs.config import CfgNode 
import torch
import causal_nf.utils.io as causal_nf_io
import time
import numpy as np
# from causal_nf.preparators.FF_preparator import FFPreparator
from causal_nf.config import cfg
from tqdm import tqdm

/Users/llaurabat/micromamba/envs/causal-flows/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
ate_folders = ['output_FF_ate1_anFalse', 'output_FF_ate5_anFalse'] #, 'output_FF_ate5'] #output_FF_ate5
models = ['model_3']#,'model_2']#, 'model_3']#'model_2', 'model_3'
model_map = {'model_1':'M1', 'model_2':'M2', 'model_3':'M3'}
versions = ['v_0', 'v_1','v_2','v_3','v_4',]#'v_0','v_1','v_2','v_3','v_4',
all_ates = {'output_FF_ate1_anFalse': {'model_1':{'v_0':[],'v_1':[],'v_2':[],'v_3':[],'v_4':[],},
                              'model_2':{'v_0':[],'v_1':[],'v_2':[],'v_3':[],'v_4':[],},
                              'model_3':{'v_0':[],'v_1':[],'v_2':[],'v_3':[],'v_4':[],},},
            'output_FF_ate5_anFalse': {'model_1':{'v_0':[],'v_1':[],'v_2':[],'v_3':[],'v_4':[],},
                              'model_2':{'v_0':[],'v_1':[],'v_2':[],'v_3':[],'v_4':[],},
                              'model_3':{'v_0':[],'v_1':[],'v_2':[],'v_3':[],'v_4':[],},} } 

In [8]:
for ate_folder in ate_folders:
    for model in models:
        for version in versions:
            
            base_path = f'./{ate_folder}/{model}/{version}/'

            try:
                directories = [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]
            except:    
                print(f"No directories found in {base_path}. Skip. ")
                continue
            
            # Assuming there's only one directory in A/B/C
            if len(directories)==1:
                file_dir = os.path.join(base_path, directories[0])
                print(file_dir)
            elif len(directories)>1:
                print(f"Multiple directories found: {directories} in base path {base_path}")
            
            
            # file_dir = './output_FF_ate1/model_1/v_0/2e684ef6533a11ef991cacde48001122'
            ckpt_file= f"{file_dir}/last.ckpt"
            
            args_list = []
            args =  CfgNode({'config_file': f'{file_dir}/wandb_local/config_local.yaml', 
                             'config_default_file': f'{file_dir}/wandb_local/default_config.yaml', 
                             'project': None, 'wandb_mode': 'disabled', 'wandb_group': None, 
                             'load_model': f'{file_dir}', 'delete_ckpt': False})
            config = causal_nf_config.build_config(
                config_file=args.config_file,
                args_list=args_list,
                config_default_file=args.config_default_file,
            )
            causal_nf_config.assert_cfg_and_config(cfg, config)

            if model=='model_1':
                from causal_nf.preparators.FF_preparator_M1 import FFPreparatorM1 
                preparator = FFPreparatorM1.loader(cfg.dataset)
                index = 4
            if model=='model_2':
                from causal_nf.preparators.FF_preparator_M2 import FFPreparatorM2
                preparator = FFPreparatorM2.loader(cfg.dataset)
                index = 4
            if model=='model_3':
                from causal_nf.preparators.FF_preparator_M3 import FFPreparatorM3
                preparator = FFPreparatorM3.loader(cfg.dataset)
                index = 10
            
            
            preparator.prepare_data()
            
            # IMPORTANT: insert ckpt file otherwise it loads anyway bt something wrong
            model_lightning = causal_nf_train.load_model(cfg=cfg, preparator=preparator, ckpt_file=ckpt_file)
            _ = model_lightning.model.eval()
            
            
            a = 1
            b = 0
            rounds = 5
            
            
            ates = []
            for seed in tqdm(range(rounds)):
                torch.random.manual_seed(seed + 1)
                ate = model_lightning.model.compute_ate(
                    index,
                    a=a,
                    b=b,
                    num_samples=10000,
                    scaler=preparator.scaler_transform,
                )
                ates.append(ate[-1])
            ates = np.array(ates)
            all_ates[ate_folder][model][version] = ates
            
            ate_mean = ates.mean()
            ate_stddev = ates.std()
            print(f"Folder {ate_folder} {model} {version}: Mean ATE over {rounds} rounds = ", ate_mean)
            print(f"Folder {ate_folder} {model} {version}: Stddev ATE over {rounds} rounds = ", ate_stddev)
            

./output_FF_ate1_anFalse/model_3/v_0/14d437f48ccc11efa0f1acde48001122

Preparing data...

Preparing data...

Preparing data...
NSF(
  (transforms): ModuleList(
    (0): MaskedAutoregressiveTransform(
      (base): MonotonicRQSTransform(bins=8)
      (order): [0, 1, 2, 3, 4, ..., 7, 8, 9, 10, 11]
      (hyper): MaskedMLP(
        (0): MaskedLinear(in_features=12, out_features=32, bias=True)
        (1): ELU(alpha=1.0)
        (2): MaskedLinear(in_features=32, out_features=32, bias=True)
        (3): ELU(alpha=1.0)
        (4): MaskedLinear(in_features=32, out_features=32, bias=True)
        (5): ELU(alpha=1.0)
        (6): MaskedLinear(in_features=32, out_features=276, bias=True)
      )
    )
  )
  (base): DiagNormal(loc: torch.Size([12]), scale: torch.Size([12]))
)
[INFO] Loading <class 'causal_nf.models.causal_nf.CausalNFightning'> from ./output_FF_ate1_anFalse/model_3/v_0/14d437f48ccc11efa0f1acde48001122/last.ckpt
scaler_transform StandardTransform(shift=tensor([3.6822, 3.6857, 3.67

100%|█████████████████████████████████████████████████████████████████████| 5/5 [00:07<00:00,  1.57s/it]


Folder output_FF_ate1_anFalse model_3 v_0: Mean ATE over 5 rounds =  0.78236663
Folder output_FF_ate1_anFalse model_3 v_0: Stddev ATE over 5 rounds =  0.012475718
./output_FF_ate1_anFalse/model_3/v_1/89a5068e8ccd11efaffeacde48001122

Preparing data...

Preparing data...

Preparing data...
NSF(
  (transforms): ModuleList(
    (0): MaskedAutoregressiveTransform(
      (base): MonotonicRQSTransform(bins=8)
      (order): [0, 1, 2, 3, 4, ..., 7, 8, 9, 10, 11]
      (hyper): MaskedMLP(
        (0): MaskedLinear(in_features=12, out_features=32, bias=True)
        (1): ELU(alpha=1.0)
        (2): MaskedLinear(in_features=32, out_features=32, bias=True)
        (3): ELU(alpha=1.0)
        (4): MaskedLinear(in_features=32, out_features=32, bias=True)
        (5): ELU(alpha=1.0)
        (6): MaskedLinear(in_features=32, out_features=276, bias=True)
      )
    )
  )
  (base): DiagNormal(loc: torch.Size([12]), scale: torch.Size([12]))
)
[INFO] Loading <class 'causal_nf.models.causal_nf.CausalNFig

100%|█████████████████████████████████████████████████████████████████████| 5/5 [00:08<00:00,  1.61s/it]


Folder output_FF_ate1_anFalse model_3 v_1: Mean ATE over 5 rounds =  0.9481517
Folder output_FF_ate1_anFalse model_3 v_1: Stddev ATE over 5 rounds =  0.013156179
./output_FF_ate1_anFalse/model_3/v_2/2d66d4148cd311efadeaacde48001122

Preparing data...

Preparing data...

Preparing data...
NSF(
  (transforms): ModuleList(
    (0): MaskedAutoregressiveTransform(
      (base): MonotonicRQSTransform(bins=8)
      (order): [0, 1, 2, 3, 4, ..., 7, 8, 9, 10, 11]
      (hyper): MaskedMLP(
        (0): MaskedLinear(in_features=12, out_features=32, bias=True)
        (1): ELU(alpha=1.0)
        (2): MaskedLinear(in_features=32, out_features=32, bias=True)
        (3): ELU(alpha=1.0)
        (4): MaskedLinear(in_features=32, out_features=32, bias=True)
        (5): ELU(alpha=1.0)
        (6): MaskedLinear(in_features=32, out_features=276, bias=True)
      )
    )
  )
  (base): DiagNormal(loc: torch.Size([12]), scale: torch.Size([12]))
)
[INFO] Loading <class 'causal_nf.models.causal_nf.CausalNFigh

...

Folder output_FF_ate5_anFalse model_3 v_2: Mean ATE over 5 rounds =  4.8931274
Folder output_FF_ate5_anFalse model_3 v_2: Stddev ATE over 5 rounds =  0.013791771
./output_FF_ate5_anFalse/model_3/v_3/9d61122e8cd911efaaeeacde48001122

Preparing data...

Preparing data...

Preparing data...
NSF(
  (transforms): ModuleList(
    (0): MaskedAutoregressiveTransform(
      (base): MonotonicRQSTransform(bins=8)
      (order): [0, 1, 2, 3, 4, ..., 7, 8, 9, 10, 11]
      (hyper): MaskedMLP(
        (0): MaskedLinear(in_features=12, out_features=32, bias=True)
        (1): ELU(alpha=1.0)
        (2): MaskedLinear(in_features=32, out_features=32, bias=True)
        (3): ELU(alpha=1.0)
        (4): MaskedLinear(in_features=32, out_features=32, bias=True)
        (5): ELU(alpha=1.0)
        (6): MaskedLinear(in_features=32, out_features=276, bias=True)
      )
    )
  )
  (base): DiagNormal(loc: torch.Size([12]), scale: torch.Size([12]))
)
[INFO] Loading <class 'causal_nf.models.causal_nf.CausalNFigh

100%|█████████████████████████████████████████████████████████████████████| 5/5 [00:09<00:00,  1.94s/it]


Folder output_FF_ate5_anFalse model_3 v_3: Mean ATE over 5 rounds =  4.549229
Folder output_FF_ate5_anFalse model_3 v_3: Stddev ATE over 5 rounds =  0.014754674
./output_FF_ate5_anFalse/model_3/v_4/43d7da4c8cdb11ef9a40acde48001122

Preparing data...

Preparing data...

Preparing data...
NSF(
  (transforms): ModuleList(
    (0): MaskedAutoregressiveTransform(
      (base): MonotonicRQSTransform(bins=8)
      (order): [0, 1, 2, 3, 4, ..., 7, 8, 9, 10, 11]
      (hyper): MaskedMLP(
        (0): MaskedLinear(in_features=12, out_features=32, bias=True)
        (1): ELU(alpha=1.0)
        (2): MaskedLinear(in_features=32, out_features=32, bias=True)
        (3): ELU(alpha=1.0)
        (4): MaskedLinear(in_features=32, out_features=32, bias=True)
        (5): ELU(alpha=1.0)
        (6): MaskedLinear(in_features=32, out_features=276, bias=True)
      )
    )
  )
  (base): DiagNormal(loc: torch.Size([12]), scale: torch.Size([12]))
)
[INFO] Loading <class 'causal_nf.models.causal_nf.CausalNFight

100%|█████████████████████████████████████████████████████████████████████| 5/5 [00:10<00:00,  2.10s/it]

Folder output_FF_ate5_anFalse model_3 v_4: Mean ATE over 5 rounds =  4.6792707
Folder output_FF_ate5_anFalse model_3 v_4: Stddev ATE over 5 rounds =  0.013229644


In [9]:
all_ates

{'output_FF_ate1_anFalse': {'model_1': {'v_0': [],
   'v_1': [],
   'v_2': [],
   'v_3': [],
   'v_4': []},
  'model_2': {'v_0': [], 'v_1': [], 'v_2': [], 'v_3': [], 'v_4': []},
  'model_3': {'v_0': array([0.78446054, 0.75983334, 0.78506064, 0.7983589 , 0.7841196 ],
         dtype=float32),
   'v_1': array([0.95228267, 0.9230776 , 0.95219433, 0.9621017 , 0.9511019 ],
         dtype=float32),
   'v_2': array([0.97803104, 0.9535986 , 0.97753763, 0.9896543 , 0.97988534],
         dtype=float32),
   'v_3': array([0.8294792 , 0.80320954, 0.8300955 , 0.8426051 , 0.83155525],
         dtype=float32),
   'v_4': array([0.8377652 , 0.81177163, 0.8351793 , 0.84888005, 0.83626986],
         dtype=float32)}},
 'output_FF_ate5_anFalse': {'model_1': {'v_0': [],
   'v_1': [],
   'v_2': [],
   'v_3': [],
   'v_4': []},
  'model_2': {'v_0': [], 'v_1': [], 'v_2': [], 'v_3': [], 'v_4': []},
  'model_3': {'v_0': array([4.6478868, 4.626202 , 4.654108 , 4.6615086, 4.6430044],
         dtype=float32),
   'v_1

In [13]:
all_ates

{'output_FF_ate1_anTrue': {'model_1': {'v_0': [],
   'v_1': [],
   'v_2': [],
   'v_3': [],
   'v_4': []},
  'model_2': {'v_0': [], 'v_1': [], 'v_2': [], 'v_3': [], 'v_4': []},
  'model_3': {'v_0': array([0.6050204 , 0.58129156, 0.6107422 , 0.62358487, 0.60808086],
         dtype=float32),
   'v_1': array([0.82263505, 0.7944881 , 0.8262924 , 0.83440936, 0.8209133 ],
         dtype=float32),
   'v_2': [],
   'v_3': [],
   'v_4': []}},
 'output_FF_ate5_anTrue': {'model_1': {'v_0': [],
   'v_1': [],
   'v_2': [],
   'v_3': [],
   'v_4': []},
  'model_2': {'v_0': [], 'v_1': [], 'v_2': [], 'v_3': [], 'v_4': []},
  'model_3': {'v_0': array([4.3052893, 4.263579 , 4.2991686, 4.309739 , 4.297138 ],
         dtype=float32),
   'v_1': array([4.4943237, 4.4393473, 4.4881344, 4.4939632, 4.48355  ],
         dtype=float32),
   'v_2': [],
   'v_3': [],
   'v_4': []}}}

In [12]:
all_ates_stats = {'output_FF_ate1_anFalse': {'model_1':{'mean':[], 'std':[]}, 'model_2':{'mean':[], 'std':[]}, 'model_3':{'mean':[], 'std':[]}, },
                 'output_FF_ate5_anFalse': {'model_1':{'mean':[], 'std':[]}, 'model_2':{'mean':[], 'std':[]}, 'model_3':{'mean':[], 'std':[]}, } } 


In [13]:
for ate_folder in ate_folders:
    for model in models:
        ates_model = []
        for version in versions:
            ates_v = all_ates[ate_folder][model][version]
            ates_model += list(ates_v)
        print(ates_model)
        all_ates_stats[ate_folder][model]['mean'] = np.array(ates_model).mean()
        all_ates_stats[ate_folder][model]['std'] = np.array(ates_model).std()
        
        

[0.78446054, 0.75983334, 0.78506064, 0.7983589, 0.7841196, 0.95228267, 0.9230776, 0.95219433, 0.9621017, 0.9511019, 0.97803104, 0.9535986, 0.97753763, 0.9896543, 0.97988534, 0.8294792, 0.80320954, 0.8300955, 0.8426051, 0.83155525, 0.8377652, 0.81177163, 0.8351793, 0.84888005, 0.83626986]
[4.6478868, 4.626202, 4.654108, 4.6615086, 4.6430044, 4.9039855, 4.8756924, 4.9064736, 4.9126663, 4.9026966, 4.89433, 4.869237, 4.8952737, 4.9124336, 4.8943634, 4.552432, 4.5224085, 4.555295, 4.567172, 4.5488377, 4.682127, 4.65575, 4.6848197, 4.6960125, 4.677644]


In [14]:
all_ates_stats

{'output_FF_ate1_anFalse': {'model_1': {'mean': [], 'std': []},
  'model_2': {'mean': [], 'std': []},
  'model_3': {'mean': 0.8735243, 'std': 0.075903855}},
 'output_FF_ate5_anFalse': {'model_1': {'mean': [], 'std': []},
  'model_2': {'mean': [], 'std': []},
  'model_3': {'mean': 4.7336946, 'std': 0.1404637}}}